In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers


In [2]:
# Generator network
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(28*28, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Discriminator network
def build_discriminator(input_shape=(28, 28, 1)):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model


In [3]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model


In [4]:
# Define constants
latent_dim = 100
input_shape = (28, 28, 1)

# Build and compile the discriminator
discriminator = build_discriminator(input_shape)
discriminator.compile(optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim)

# Build and compile the GAN model
gan = build_gan(generator, discriminator)
gan.compile(optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')


In [5]:
# Load MNIST dataset
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=-1)

# Define constants
batch_size = 32
epochs = 10000

# Training loop
for epoch in range(epochs):
    # Train discriminator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_images = generator.predict(noise)
    real_images = X_train[np.random.randint(0, X_train.shape[0], batch_size)]
    X = np.concatenate([real_images, fake_images])
    y = np.zeros(2*batch_size)
    y[:batch_size] = 1
    d_loss = discriminator.train_on_batch(X, y)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    y = np.ones(batch_size)
    g_loss = gan.train_on_batch(noise, y)

    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

    # Save generated images
    if epoch % 1000 == 0:
        noise = np.random.normal(0, 1, (10, latent_dim))
        generated_images = generator.predict(noise)
        for i in range(generated_images.shape[0]):
            plt.imshow(generated_images[i, :, :, 0], cmap='gray')
            plt.axis('off')
            plt.savefig(f"generated_image_{epoch}_{i}.png")
            plt.close()


1/1 [==============================] - 0s 186ms/step
Epoch: 0, D Loss: 0.6261216402053833, G Loss: 0.5403119921684265
1/1 [==============================] - 0s 16ms/step
Epoch: 100, D Loss: 0.41783249378204346, G Loss: 0.6987553834915161
1/1 [==============================] - 0s 16ms/step
Epoch: 200, D Loss: 0.38936352729797363, G Loss: 0.9062672257423401
1/1 [==============================] - 0s 16ms/step
Epoch: 300, D Loss: 0.38769930601119995, G Loss: 0.8294826745986938
1/1 [==============================] - 0s 16ms/step
Epoch: 400, D Loss: 0.4742487668991089, G Loss: 0.8944968581199646
1/1 [==============================] - 0s 13ms/step
Epoch: 500, D Loss: 0.5012981295585632, G Loss: 0.822527289390564
1/1 [==============================] - 0s 0s/step
Epoch: 600, D Loss: 0.5559282302856445, G Loss: 0.8550869226455688
1/1 [==============================] - 0s 16ms/step
Epoch: 700, D Loss: 0.5319809317588806, G Loss: 0.8044911623001099
1/1 [==============================] - 0s 9ms/ste

KeyboardInterrupt: 